# **Retrieving the state-space representation from Dynare**

**Real Business Cycle Model** \\
João Ricardo Costa Filho \\
joaocostafilho.com

## Install Octave and Dynare in the "virtual" machine.

In [1]:
!apt-get install octave &> /dev/null

In [2]:
!apt-get install dynare &> /dev/null

## Create the 'RBC.mod' file

In [3]:
%%writefile RBC.mod

% Code by Erci Sims: https://www3.nd.edu/~esims1/grad_macro_17.html > Using Dynare: pdf
% Adapted by João Costa-Filho

var y I k a c w R r;

varexo e;

parameters alpha beta delta rho sigma sigmae;

alpha = 1/3;
beta = 0.99;
delta = 0.025;
rho = 0.95;
sigma = 1;
sigmae = 0.01;

model;

exp( c )^( -sigma ) = beta * exp( c(+1) )^( -sigma ) * ( alpha * exp( a(+1) ) * exp( k )^( alpha - 1 ) + ( 1 - delta ) );
exp(y) = exp(a)*exp(k(-1))^(alpha);
exp(k) = exp(a)*exp(k(-1))^(alpha) - exp(c) + (1-delta) * exp(k(-1));
a = rho*a(-1) + e;
exp(y) = exp(c) + exp(I);
exp(c)^(-sigma) = beta*exp(c(+1))^(-sigma)*(1+r);
exp(R) = alpha*exp(a)*exp(k(-1))^(alpha-1);
exp(w) = (1-alpha)*exp(a)*exp(k(-1))^(alpha);
end;

initval;
k = log(30);
y = log(3);
c = log(2.5);
I = log(0.5);
a = 0;
r = (1/beta) - 1;
R = log((1/beta) - (1-delta));
w = log(1);
end;

shocks;
var e = 1;
end;

steady;

stoch_simul(order=1,irf=20, nograph);

Writing RBC.mod


In [4]:
%%writefile RBC.m
dynare RBC.mod


Writing RBC.m


## Solve the model with Dynare

In [5]:
!octave RBC.m &> /dev/null

In [6]:
!dynare RBC.mod &> /dev/null

## Create the 'simulations.m' file and run the IRF (figure 1) and stochastic simulations (figure 2)

In [16]:
%%writefile simulations.m


%% Housekeeping

close all;
clear;

%% solve the model

dynare RBC;

%clearvars -except oo_ M_ model;

%% State Space

% state space representation: 
% S_t = A * S_{t-1} + B * e_{t}, 
% X_t = C * S_{t-1} + D * e_{t};

%initiate the matrices

% baseline model
A = [];
B = [];
C = [];
D = [];

display(M_.state_var)

%state variables
state = M_.state_var';

%control variables
control = [1:1:size(oo_.dr.ghu,1)]';

%remove state variables from control variables vector
for j = 1:size(state,1)
    
    control( control == state(j) ) = [];

end

A = [oo_.dr.ghx( oo_.dr.inv_order_var( state ), : ) ];

B = [oo_.dr.ghu( oo_.dr.inv_order_var( state ), : ) ];

C = [oo_.dr.ghx( oo_.dr.inv_order_var( control ), : ) ];

D = [oo_.dr.ghu( oo_.dr.inv_order_var( control ), : ) ];

%exogenous variables names
exogenous = M_.exo_names;  

%endogenous variables names
endogenous = M_.endo_names;

%% Impulse-Response Functions (IRF)

horizon = 20;
Sirf = zeros( size(state,1), horizon );   % States IRF matrix
Xirf = zeros( size(control,1), horizon);  % Control IRF matrix

%variables names
S_variables_names = M_.endo_names(state);
X_variables_names = M_.endo_names(control);

%vector of shocks
shocks = [ 1 ];      % TFP shock;

Sirf(:,1) = B * shocks;  % The initial value (at the time of the shocks) for the state variables
Xirf(:,1) = D * shocks;  % The initial value (at the time of the shocks) for the control variables

for j = 2:horizon
    Sirf(:,j) = A * Sirf(:,j-1);
    Xirf(:,j) = C * Sirf(:,j-1);
end

%% Graphs

%TFP shock;

figure('Position', get(0, 'Screensize'))
subplot(2,2,1)
plot(1:horizon,Xirf( find( X_variables_names == "y") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 ) 
axis("square")
xlim([1 horizon])
title( 'Output (%)' )
hold on

subplot(2,2,2)
plot(1:horizon, Xirf( find( X_variables_names == "c") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 ) 
axis("square")
xlim([1 horizon])
title('Consumption (%)')
hold on

subplot(2,2,3)
plot(1:horizon, Xirf( find( X_variables_names == "w") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 )
axis("square")
xlim([1 horizon])
title('Wages (%)')
hold on

subplot(2,2,4)
plot(1:horizon, Xirf( find( X_variables_names == "r") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 )
axis("square")
xlim([1 horizon])
title('Real interest rate (p.p.)')
hold off
exportgraphics(gcf,'figure1.jpg','Resolution',300)

%% Simulations - Sequence of random shocks

e = [ 1 0.5 -0.25 -0.75 0.25 0.17 -0.1 0 ]'; %TFP shock;

horizon = size(e,1)+1;

shocks_names = M_.exo_names;

shocks = zeros(M_.exo_nbr, horizon);

shocks( find( shocks_names == "e"),2:horizon) = e;
    
Ssim = zeros( size(state,1), horizon);
Xsim = zeros( size(control,1), horizon);

for j = 2:horizon
    
    Ssim(:,j) = A * Ssim(:,j-1) + B * shocks(:,j);
    Xsim(:,j) = C * Ssim(:,j-1) + D * shocks(:,j);
    
end


%% Graphs


% TFP shock;

figure('Position', get(0, 'Screensize'))
subplot(2,2,1)
plot(1:horizon,Xsim( find( X_variables_names == "y") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 ) 
axis("square")
xlim([1 horizon])
title( 'Output (%)' )
hold on

subplot(2,2,2)
plot(1:horizon, Xsim( find( X_variables_names == "c") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 ) 
axis("square")
xlim([1 horizon])
title('Consumption (%)')
hold on

subplot(2,2,3)
plot(1:horizon, Xsim( find( X_variables_names == "w") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 )
axis("square")
xlim([1 horizon])
title('Wages (%)')
hold on

subplot(2,2,4)
plot(1:horizon, Xsim( find( X_variables_names == "r") ,:), 'Color',[0.6350 0.0780 0.1840], 'Linewidth', 2 )
axis("square")
xlim([1 horizon])
title('Real interest rate (p.p.)')
hold off
exportgraphics(gcf,'figure2.jpg','Resolution',300)



Overwriting simulations.m


In [17]:
!octave simulations.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features

otherwise m files will be used instead of precompiled mex files and some features, like solution
of models approximated at third order, will not be available.
    dynare_config at line 97 column 5
    dynare at line 71 column 12
    simulations at line 10 column 1

    dynare_config at line 140 column 1
    dynare at line 71 column 12
    simulations at line 10 column 1
    dynare_config at line 140 column 1
    dynare at line 71 column 12
    simulations at line 10 column 1

Configuring Dynare ...
[mex] Generalized QZ.
[mex] Sylvester equation solution.
[mex] Kronecker products.
[mex] Sparse kronecker products.
[mex] Local state space iteration (second order).
[mex] Bytecode evaluation.
[mex] k-order perturbation solver.
[mex] k-order solution simulation.
[mex] Quasi Monte-Carlo sequence (Sobol).
[mex] Markov Switching SBVAR.

Using 64-bit preprocessor
Starting Dynare (version 4.5.4).
Starting preprocessi